## 세팅

In [45]:
!pip install konlpy

   ---------------------------------------- 0.0/19.4 MB ? eta -:--:--
   ---- ----------------------------------- 2.4/19.4 MB 12.2 MB/s eta 0:00:02
   -------- ------------------------------- 4.2/19.4 MB 10.1 MB/s eta 0:00:02
   ----------- ---------------------------- 5.5/19.4 MB 8.8 MB/s eta 0:00:02
   -------------- ------------------------- 7.1/19.4 MB 8.6 MB/s eta 0:00:02
   ------------------- -------------------- 9.4/19.4 MB 9.3 MB/s eta 0:00:02
   ------------------------ --------------- 11.8/19.4 MB 9.7 MB/s eta 0:00:01
   ---------------------------- ----------- 13.6/19.4 MB 9.6 MB/s eta 0:00:01
   ------------------------------- -------- 15.5/19.4 MB 9.5 MB/s eta 0:00:01
   ------------------------------------ --- 17.8/19.4 MB 9.9 MB/s eta 0:00:01
   ---------------------------------------- 19.4/19.4 MB 9.7 MB/s eta 0:00:00


In [46]:
!pip install JPype1

In [65]:
!pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
    --------------------------------------- 2.4/124.9 MB 12.2 MB/s eta 0:00:11
   - -------------------------------------- 4.7/124.9 MB 11.9 MB/s eta 0:00:11
   -- ------------------------------------- 7.1/124.9 MB 11.8 MB/s eta 0:00:10
   --- ------------------------------------ 9.4/124.9 MB 11.7 MB/s eta 0:00:10
   --- ------------------------------------ 11.8/124.9 MB 11.7 MB/s eta 0:00:10
   ---- ----------------------------------- 14.2/124.9 MB 11.9 MB/s eta 0:00:10
   ----- ---------------------------------- 16.5/124.9 MB 11.8 MB/s eta 0:00:10
   ------ --------------------------------- 18.9/124.9 MB 11.8 MB/s eta 0:00:09
   ------ --------------------------------- 21.2/124.9 MB 11.8 MB/s eta 0:00:09
   ------- -------------------------------- 23.6/124.9 MB 11.8 MB/s eta 0:00:09
   -------- ------------------------------- 26.0/124.9 MB 11.7 MB/s eta 0:00:09
   --------- ------------------------------ 28.3/124.

In [122]:
!pip install scikit-learn

## 함수

In [6]:
# get_driver()
# 사용 전제 조건 : Users 폴더에 버전에 맞는 chromedriver.exe를 다운받아주세요
# 기능 : driver를 반환합니다
# 리턴값 : driver
def get_driver():
    CHROME_DRIVER_PATH = "C:/Users/chromedriver.exe"    # (절대경로) Users 폴더에 chromedriver.exe를 설치했음
    options = webdriver.ChromeOptions()                 # 옵션 선언
    # [옵션 설정]
    options.add_argument("--start-maximized")         # 창이 최대화 되도록 열리게 한다.
    # options.add_argument("--headless")                  # 창이 없이 크롬이 실행이 되도록 만든다
    options.add_argument("disable-infobars")            # 안내바가 없이 열리게 한다.
    options.add_argument("disable-gpu")                 # 크롤링 실행시 GPU를 사용하지 않게 한다.
    options.add_argument("--disable-dev-shm-usage")     # 공유메모리를 사용하지 않는다
    options.add_argument("--blink-settings=imagesEnabled=false")    # 이미지 로딩 비활성화
    options.add_argument('--disk-cache-dir=/path/to/cache-dir')     # 캐시 사용 활성화
    options.page_load_strategy = 'none'             # 전체 페이지가 완전히 로드되기를 기다리지 않고 다음 작업을 수행 (중요)
    options.add_argument('--log-level=3')           # 웹 소켓을 통한 로그 메시지 비활성화
    options.add_argument('--no-sandbox')            # 브라우저 프로파일링 비활성화
    options.add_argument('--disable-plugins')       # 다양한 플러그인 및 기능 비활성화
    options.add_argument('--disable-extensions')    # 다양한 플러그인 및 기능 비활성화
    options.add_argument('--disable-sync')          # 다양한 플러그인 및 기능 비활성화
    driver = webdriver.Chrome(CHROME_DRIVER_PATH, options=options)
    return driver


# 기능 : url을 받아 requests를 사용하여 soup를 리턴하는 함수입니다
def get_soup(url, time_sleep=0, max_retries=600):
    try:
        if max_retries <= 0:
            print("[최대 재시도 횟수 초과 : get_soup()]")
            return None
        with requests.Session() as session:
            response = session.get(url, headers=headers)
            time.sleep(time_sleep)
        soup = BeautifulSoup(response.text, "html.parser")
        if len(soup) == 0:  # 불러오는데 실패하면
            print(f"[soup 로딩 실패, 반복] [get_soup(time_sleep=1)]")
            soup = get_soup(url, 1)
    except Exception as e:
        print(f"[오류 발생, 반복] [get_soup(time_sleep=1)] ", e)
        soup = get_soup(url, 1, max_retries-1)
    return soup



# 전처리 함수
def extract_korean(text):
    korean_pattern = re.compile(r'[가-힣\s]+')  # 한글 정규 표현식
    matches = korean_pattern.findall(text)
    extracted_text = "".join(matches)

    processed_text = re.sub(r'\s{2,}', ' ', extracted_text)     # 2개 이상의 공백을 1개로 대체
    return processed_text.strip()


# url 받아 html의 한글만 추출하는 함수
def get_html_korean(input_url, mode='selenium'):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36",
        "Connection": "keep-alive",
        "Cache-Control": "max-age=0",
        "sec-ch-ua-mobile": "?0",
        "DNT": "1",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-User": "?1",
        "Sec-Fetch-Dest": "document",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "ko-KR,ko;q=0.9"
    }
    html = ''
    if mode == 'bs':
        soup = get_soup(input_url)
        html = str(soup)
        
    elif mode == 'selenium':
        driver = get_driver()
        driver.get(input_url)
        html = driver.page_source
        driver.quit()
    
    html_korean = extract_korean(html)
    
    return html_korean


# url 리스트를 받아서, html의 한글만 추출하여 df 만들고 반환
def make_df_from_url_list(url_list, mode='selenium'):
    data = []
    for index, url in enumerate(url_list):
        print(f'{index+1} / {len(url_list)}')
        html_korean = get_html_korean(url, mode)
        row = [url, html_korean]
        data.append(row)
    
    df = pd.DataFrame(data, columns=["url", "html_korean"])
    return df

## 코드

### 세팅

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import re
import pandas as pd
import requests
import time

In [3]:
# 도박 사이트 url

gambling_url_list = [
    'https://smtb-0015.com/?ref=7731',
    'https://winner1.top/?regcode=7745',
    'https://dgg-0077.com/?ref=7731',
    'https://www.me-44.com/',
    'https://damwiki1.com/',
    'https://www.into88cc.com/?affiliateCode=diu69',
    'https://bwbw88.com/',
    'https://lovecasino.live/',
    'https://1bet1.pw/bbs/board.php?bo_table=a25&regcode=7336',
    'https://lula.sc/Main?agentCode=9966',
    'https://gob-102.com/?ref=2099',
    'https://kcy-01.com/?ref=7731',
    'http://lf-1122.com/login.asp',
    'https://jb-xy.com/login.html',
    'https://cco42.com/register/?aff_regis_code=5555',
    'https://tgtg-1144.com/',
    'http://tm-4477.com/login.asp',
    'https://nd-0001.com/',
    'http://up-5577.com/login.asp',
    'https://lv-8895.com/login.asp',
    'http://core-333.com/login.asp',
    'http://nrt-1070.com/login.asp',
    'https://lg-1212.com/',
    'http://tenca-7777.com/login.asp',
    'http://bolt-1818.com/login.asp',
    'http://snc-2001.com/login.asp',
    'https://boss-7576.com/',
    'http://gn3300.com/login.asp',
    'https://pan-1010.com/',
    'https://hero-7733.com/',
    
    'http://bolt-1818.com/login.asp',
    'https://eve1.bet/?referral=9090',
    'https://yes2.bet/?referral=9090',
    'https://baro.bet/sports/prematch?referral=0066',
    'https://nana-mul.com/',
    'https://onetop.pw/Main',
    'http://kc-2288.com/login.asp',
    'https://onetop.pw/Main',
    'https://www.xapb32.com/?btag=2300099',
    'https://apo-66.com/front',
    'https://arr-365.com/',
    'https://wwwholdem.com/',
]

print(len(gambling_url_list))

# 중복제거
unique_gambling_url_list = set(gambling_url_list)
print(len(unique_gambling_url_list))

42
40


In [4]:
# 광고 사이트 url

ad_url_list = [
    'https://www.kr.playblackdesert.com/ko-KR/News/Detail?groupContentNo=13331&ADcampaign=ao&ADSource=gdn_cpa_newpu&ADTrackerName=keyword&ADMedium=kr_notice&ADContent=i_250221_dnseason&utm_campaign=ao&utm_source=gdn_cpa_newpu&utm_term=keyword&utm_medium=kr_notice&utm_content=i_250221_dnseason&gameCode=101&gad_source=5&gclid=EAIaIQobChMIu_ebmJ_fiwMVR1sPAh1cywfnEAEYASAAEgKDSvD_BwE',
    'https://lineagem.plaync.com/preorder/record/homecoming/index?ip=lm&cn=homecoming&cc=kr&pi=pre_order&tp=branding&src=gdn&med=pc_network_banner_ua&val=perfect&gad_source=5&gclid=EAIaIQobChMI3fLmn6DfiwMV9cgWBR1aYDAxEAEYASAAEgJYk_D_BwE',
    'https://h5.g123.jp/game/arifure?lang=ko&platform=auto&utm_source=google&utm_campaign=arifure_ko_NewDisplayAttributionOptimize_adn_all&utm_adgroup=cr_normal_pay_similar_alltitle_pc&utm_content=arifure_ja_ai-wh-wnolg-3e5c96_728-90.png&gad_source=5&gclid=Cj0KCQiA8fW9BhC8ARIsACwHqYpNqAlw939wS96Q7_J0ibEpFHrJVcobQAoIouPghzmBbM9uTm0n2JkaAihpEALw_wcB',
    'https://lineagem.plaync.com/preorder/record/homecoming/index?ip=lm&cn=homecoming&cc=kr&pi=pre_order&tp=branding&src=gdn&med=pc_network_banner_ua&val=perfect&gad_source=5&gclid=EAIaIQobChMIwM_Y5qHfiwMVxMoWBR1vHCybEAEYASAAEgIAXvD_BwE',
    'https://hanghae99.spartacodingclub.kr/plus/be?utm_source=google&utm_medium=display&utm_campaign=20738103311&utm_content=156802623562&utm_term=705400258540&gad_source=5&gclid=EAIaIQobChMIkZCah6LfiwMV4yV7Bx3SShKbEAEYASAAEgIIgvD_BwE',
    'https://www.skyscanner.co.kr/transport/flights/SELA/FUK/cheapest-flights-from-%EC%84%9C%EC%9A%B8-%EB%8F%84%EC%B0%A9-%EC%A0%80%EB%A0%B4%ED%95%9C-%ED%95%AD%EA%B3%B5%EA%B6%8C-%ED%9B%84%EC%BF%A0%EC%98%A4%EC%B9%B4.html?oym=2503&iym=2504&preferDirects=false&qp_prevPrice=121645&qp_prevProvider=mas_adfeeds&qp_prevCurrency=KRW&utm_medium=display&utm_source=criteo&utm_campaign=kr-flights-midfunnel-feed&utm_content=feed&utm_term=317702&AssociateID=DIS_FLI_00053_00000&campaign_id=298050&adgroupid=317702&click_timestamp=1740501687&utm_id=317702&cto_pld=sxASONdTAABSx7Lj_-59UQ&selectedoday=01&selectediday=01',
    'https://digitalshinhansec.com/global/event01/?utm_source=pfm_gdn&utm_medium=da&utm_campaign=overseasstockwlc_pc&utm_content=gdn_kwcompetitorbrand&utm_term=729413447671&gad_source=5',
    'https://lovelpia.com/intro_landing/preview_hangtoon3?p_id=webgdn289&gad_source=5&gclid=Cj0KCQiA8fW9BhC8ARIsACwHqYrb60T0lH61vEXHQiosCPLLqY2-T4WQFyeH9YDdho3CCHDQtc9TnoQaAupKEALw_wcB',
    'https://lovelpia.com/intro_landing/tpo2?p_id=webgdn285&gad_source=5&gclid=Cj0KCQiA8fW9BhC8ARIsACwHqYoBleCLJqkRI1fuabnNVVX-1CxTC0x3RyNiTJUv03k1rjjXGo9frigaAgYjEALw_wcB',
    'https://menovel.com/sub/view.php?cpid=gdnnovel3&itemNo=72&itemIdx=210',
    'https://www.miral.org/bbs/view.asp?bbsIdx=90384&utm_source=gdn_responisve&utm_medium=banner&utm_campaign=%ED%95%98%EB%9E%91&utm_content=%ED%95%98%EB%9E%91&gad_source=5&gclid=Cj0KCQiA8fW9BhC8ARIsACwHqYoM0EA22bWCa-kIi6x9hzdSe9VIl-X_NbUZiOoiSfZKI2ks8Q2upn4aAupGEALw_wcB',
    'https://www.kcar.com/bc/detail/carInfoDtl?i_sCarCd=EC61104501&gad_source=5&gclid=Cj0KCQiA8fW9BhC8ARIsACwHqYroImtQxi1QhQ5KsPj_A82CNDpJi2jx3VMNPsnoezLwFlsJfeXxANAaAny1EALw_wcB',
    'https://lovelpia.com/intro_landing/sister1?p_id=webgdn300&gad_source=5&gclid=Cj0KCQiA8fW9BhC8ARIsACwHqYrvyjM1lKRCYbJw_jyxHBWCevlMb5pRcSaTa7atTw4HMJiRH7CQbzsaAhYSEALw_wcB',
    'https://www.adobe.com/kr/creativecloud/roc/photoshop/everyone-can.html?sdid=7RZ35XBM&mv=display&mv2=display',
    'https://www.etoos.com/teacher/event/2025/02/lsh_12/?gad_source=5&gclid=Cj0KCQiA8fW9BhC8ARIsACwHqYoWqTSYMt1enr4eISMRqQ-MP3v7HuTl4bRun0eEQIq9h4Y-jBWHazAaAqArEALw_wcB',
    'https://www.bjmart.kr/shop_view/?idx=235&utm_source=taboola&utm_medium=referral&tblci=GiBbnJDDE0WqfYdoGEhzKxhEb2lhGi0xCmqEE5LzIMkuCSDd5G4oypiwgJSc65oBMIWxVQ#tblciGiBbnJDDE0WqfYdoGEhzKxhEb2lhGi0xCmqEE5LzIMkuCSDd5G4oypiwgJSc65oBMIWxVQ',
    'https://soc.seaofconquest.com/pc/pc-landing1-kr?ms=taboola&cp=MOPC_KR_Taboola_CPA_MaxCon_202501122&cp_id=43906274&ag=chosun-mobile&ag_id=1398917&ad=MO_01147A01&ad_id=4094084865&&transaction_id=GiBbnJDDE0WqfYdoGEhzKxhEb2lhGi0xCmqEE5LzIMkuCSDczW0o2_b34JC356H2ATCFsVU&auto=1&tblci=GiBbnJDDE0WqfYdoGEhzKxhEb2lhGi0xCmqEE5LzIMkuCSDczW0o2_b34JC356H2ATCFsVU#tblciGiBbnJDDE0WqfYdoGEhzKxhEb2lhGi0xCmqEE5LzIMkuCSDczW0o2_b34JC356H2ATCFsVU',
    'https://www.miral.org/bbs/view.asp?bbsIdx=77456&utm_source=gdn_responisve&utm_medium=banner&utm_campaign=%EC%A7%80%EC%9C%A0&utm_content=%EC%A7%80%EC%9C%A0&gad_source=5&gclid=Cj0KCQiA8fW9BhC8ARIsACwHqYr2Rf1xBIIdJ_8cTVxFH4aOPSjkIjCd9xBswNhHS0QMtGoUiOcz92MaAj1tEALw_wcB',
    'https://www.bjmart.kr/shop_view/?idx=235&utm_source=taboola&utm_medium=referral&tblci=GiBbnJDDE0WqfYdoGEhzKxhEb2lhGi0xCmqEE5LzIMkuCSDd5G4o-bnSvO2r2o6eATCFsVU#tblciGiBbnJDDE0WqfYdoGEhzKxhEb2lhGi0xCmqEE5LzIMkuCSDd5G4o-bnSvO2r2o6eATCFsVU',
    'https://st-pc.kingsgroupgames.com/landing-2-kr.html?ms=taboola&cp=STPC_KR_install_Video_250108&cp_id=43776314&ag=chosun-mobile&ag_id=1398917&ad=GOGEM_13623B01&transaction_id=GiBbnJDDE0WqfYdoGEhzKxhEb2lhGi0xCmqEE5LzIMkuCSCZ3WIo-rvRj-nyrtHBATCFsVU&tblci=GiBbnJDDE0WqfYdoGEhzKxhEb2lhGi0xCmqEE5LzIMkuCSCZ3WIo-rvRj-nyrtHBATCFsVU#tblciGiBbnJDDE0WqfYdoGEhzKxhEb2lhGi0xCmqEE5LzIMkuCSCZ3WIo-rvRj-nyrtHBATCFsVU',
    'https://future-news.co.kr/tatimelo/?utm_source=taboola&utm_medium=referral&tblci=GiBbnJDDE0WqfYdoGEhzKxhEb2lhGi0xCmqEE5LzIMkuCSCu_FMohZWj78TH3rgbMIWxVQ#tblciGiBbnJDDE0WqfYdoGEhzKxhEb2lhGi0xCmqEE5LzIMkuCSCu_FMohZWj78TH3rgbMIWxVQ',
    'https://h5.g123.jp/game/arifure?lang=ko&platform=auto&utm_source=google&utm_campaign=arifure_ko_NewPmaxAttributionOptimize_adn_all&utm_adgroup=normal01&gad_source=5&gclid=Cj0KCQiA8fW9BhC8ARIsACwHqYoP0U3zLqQ7TuAeXP4__U-t01LTmh-E0JvTTvBQ5WOubpBLphAi_hcaAlurEALw_wcB',
    'https://maplestory.nexon.com/Home/Main',
    'https://h5.g123.jp/game/arifure?lang=ko&platform=auto&utm_source=google&utm_campaign=arifure_ko_NewDisplayAttributionOptimize_adn_all&utm_adgroup=cr_normal_pay_similar_alltitle_pc&utm_content=arifure_ko_ai-wh-raretsu1009-bf7aa9_300-600.jpg&gad_source=5&gclid=Cj0KCQiA8fW9BhC8ARIsACwHqYqmtKgh7YX_8sBziLqih7a78CwqU4mGorC1ESz29CQBeqoL5FYuyPIaAgWfEALw_wcB',
    'https://www.kiwoometf.com/service/invest/KO04020102T?kijaGubun=10&kijaNo=365&schGubun2=0&schGubun4=1&schContent&utm_source=INVESTINGDC&utm_medium=DA&utm_campaign=KIWOOMETF_EVENT_NV_BILLBOARD&utm_content=NT_NA&utm_term=2_250214',
    'https://trc.taboola.com/seoulshinmun-seoulshinmun-desktop/log/3/click?pi=%2Fnews%2Feconomy%2F2024%2F11%2F13%2F20241113500233&ri=905337b10d17cfecc6dc699117587cfc&sd=v2_caf8281b3b380eeef70beed156a473c4_1ff2f13f-b160-400e-b3b8-74b4fd9d93b6-tucteb77654_1740502601_1740502605_CAwQjcFOGPehqPDTMiABKAMwdziKxgtA28cLSJPK8QNQ____________AVgAYBZowLya6-vc4ZfuAXABgAEA&ui=1ff2f13f-b160-400e-b3b8-74b4fd9d93b6-tucteb77654&ii=~~V1~~-5900094030570398899~~rkAi_aW8v6q7Q-ek1yG1lBKUwwpydVDKeMaZCxdWMe_noZueAsnM0UTkqRiz-o8uuKa2_rupL9I0kvx1XVufut2qKIJrSNJ_InWKPTfYcgwvFg07WPu7ezROxGyxYeqUrkKApwsDKQrOSx0uYL0pzxCZnH8w9VCdeFaxyQP1T2Pq9FM2RRDzyrp8YDLfkWEdCzetcYp999XcwyzLRVP_tZnRZkt-_ua9K0MMTzpNWfne0NHFJeL651fhbnMIa7orb2xmWtPOu-JZzvkrpQiBEA&it=text&pt=text&li=rbox-t2m&sig=26347e59689e3aec9369a0edcdd8e7508fca01149ee0&vi=1740502601975&p=techlabs-seoulplandentalimplantclinicba-sc&lti=placeholder111_var&r=575&tvi48=23830&tvi50=9540&tvi62=18148&ppb=CPwB&cpb=EhMyMDI1MDIyNS0zNC1SRUxFQVNFGLHV29MGIJz__________wEqGXNnLnRhYm9vbGFzeW5kaWNhdGlvbi5jb20yCHRyYzMwMjYwOIDclP4KQIrGC0jbxwtQk8rxA1j___________8BYwjXFhDVHxgjZGMI-f__________ARD5__________8BGAdkYwjSAxDgBhgIZGMI9___________ARD3__________8BGAlkYwjTlAEQyroBGApkYwikJxCDNRgvZGMIpZQBEJa6ARgwZGMI-jcQxEoYMmRjCPIkEJ2hARgUZGMInkEQpFYYNmRjCNwKEKAQGBZkYwiWFBCfHBgYZGMIqIUBEO6oARg6ZGMI_0YQjWYYHWRjCINuEOSNARg-ZGMI9BQQnh0YH2R4AYABAogB2qPnuQSQARyYAbi8qPDTMtsBEAHcAQ&cv=20250225-34-RELEASE&route=HK%3ASG%3AV&redir=https%3A%2F%2Fimplan.co.kr%2Flanding%2F%3Fid%3D1727233554%26utm_source%3Dtaboola%26utm_medium%3Dimage%26utm_campaign%3D%25ED%2598%259C%25ED%2583%259D%25ED%2585%258C%25EC%258A%25A4%25ED%258A%25B8%25EB%25A9%2594%25EA%25B0%2580%25EC%25A0%25A033%25EB%25A7%258C%25EC%259B%2590%26utm_content%3DW_%25ED%2583%2580%25EB%25B6%2588%25EB%259D%25BC_%25EC%259D%25B4%25EB%25AF%25B8%25EC%25A7%2580%25EC%25BA%25A0%25ED%258E%2598%25EC%259D%25B8_%25EC%2597%25B0%25EB%25A0%25B9%25ED%2583%2580%25EA%25B2%259F%25EB%25B6%2588%25EA%25B0%2580_%25EC%2584%259C%25EC%259A%25B8_%25EC%259A%25B0%25ED%258E%25B8%25EB%25B2%2588%25ED%2598%25B8%26utm_term%3D%25EA%25B8%25B0%25ED%2583%2580%25EC%2586%258C%25EA%25B5%25AC_%25EB%25AF%25B8%25EA%25B8%25B0%25EC%259E%25AC_%25EC%2597%2591%25EC%258A%25A4%25EB%25A0%2588%25EC%259D%25B4%25EB%25B9%25A8%25EA%25B0%2584%25EC%259B%2590%26tblci%3DGiBbnJDDE0WqfYdoGEhzKxhEb2lhGi0xCmqEE5LzIMkuCSC5jWso4oWtrsCHv5BQMI3BTg%23tblciGiBbnJDDE0WqfYdoGEhzKxhEb2lhGi0xCmqEE5LzIMkuCSC5jWso4oWtrsCHv5BQMI3BTg&tbu=b1d2b7b2-8ca3-4891-a1e0-b0c5afd05bb9',
    'https://lastwarriorsurvival.com/builder/landing-13-kr/?ms=taboola&cp=SSPCPK_KR_Video_20250123&cp_id=43918731&ag=seoulshinmun-seoulshinmun-desktop&ag_id=1286285&ad=SSQ_00248C04&transaction_id=GiBbnJDDE0WqfYdoGEhzKxhEb2lhGi0xCmqEE5LzIMkuCSDkp1oo5eGHhOT8jdVIMI3BTg&auto=1&tblci=GiBbnJDDE0WqfYdoGEhzKxhEb2lhGi0xCmqEE5LzIMkuCSDkp1oo5eGHhOT8jdVIMI3BTg#tblciGiBbnJDDE0WqfYdoGEhzKxhEb2lhGi0xCmqEE5LzIMkuCSDkp1oo5eGHhOT8jdVIMI3BTg',
    'https://www.ssg.com/plan/planShop.ssg?dispCmptId=6000177840&ckwhere=ssg_linkmine_cps2',
    'https://h5.g123.jp/game/arifure?lang=ko&platform=auto&utm_source=google&utm_campaign=arifure_ko_NewDisplayAttributionOptimize_adn_all&utm_adgroup=cr_normal_pay_similar_alltitle_pc&utm_content=arifure_ja_ai-sc-ssnolg-4bd259_336-280.png&gad_source=5&gclid=Cj0KCQiA8fW9BhC8ARIsACwHqYru6OR7jwVLP32b5v2F1I7trK7c_hhoGXh1kBFvVTLg_PD0NOTAfTEaAub4EALw_wcB',
    'https://mmw.tqgame.kr/simple/s_join.html?pages=7&part=mm_gdn_sub',
    
]

print(len(ad_url_list))

# 중복제거
unique_ad_url_list = set(ad_url_list)
print(len(unique_ad_url_list))

30
30


### 테스트 : html 한글만 추출

In [8]:
# test : html 한글만 추출

target_url = ad_url_list[0]
result = get_html_korean(target_url, mode='selenium')
result

'노트 시즌 에서 펼치는 캐릭터 성장 가속 검은사막 한국 검은사막 검사 게임 비디오 게임 온라인 게임 롤플레잉 오픈월드 검은사막 포럼 검사 포럼 펄어비스 꿈꿔왔던 새로운 시즌의 시작시즌 모험가님의 여정을 아낌없이 지원해 드릴 시즌 다양한 시즌 혜택과 함께 설레는 모험을 시작해 보세요 꿈결 환상마와 꿈의 모험으로검은사막에서 가장 특별한 능력을 지닌 탑승물 꿈결 환상마가 모든 모험가분들께 지급됩니다지금 검은사막 접속 후 도전과제에서 꿈결 환상마 선택 상자를 수령해 보세요꿈결 환상마 자세히 살펴보기더 높이 날아오를 모험가님의 꿈을 위해 더 빠른 성장의 꿈꿈결 둠처럼 빠른 속도로 성장할 시간시즌 서버 핫타임 노트 시즌 에서 펼치는 캐릭터 성장 가속 검은사막 한국 꿈꿔왔던 새로운 시즌의 시작시즌 모험가님의 여정을 아낌없이 지원해 드릴 시즌 다양한 시즌 혜택과 함께 설레는 모험을 시작해 보세요 꿈결 환상마와 꿈의 모험으로검은사막에서 가장 특별한 능력을 지닌 탑승물 꿈결 환상마가 모든 모험가분들께 지급됩니다지금 검은사막 접속 후 도전과제에서 꿈결 환상마 선택 상자를 수령해 보세요꿈결 환상마 자세히 살펴보기더 높이 날아오를 모험가님의 꿈을 위해 더 빠른 성장의 꿈꿈결 둠처럼 빠른 속도로 성장할 시간시즌 서버 핫타임 이전 검은사막 검은사막 모바일 붉은사막 도깨비 플랜 다음 검은사막 펄어비스 의 시작 리마스터링으로 더더욱 현실에 가까워진 모험이 당신을 기다립니다리얼함을 넘어 감동이 되는 여기는 검은사막입니다 공식 홈페이지 콘솔 공식 홈페이지 검은사막 모바일 대한민국 게임 대상 관왕검은사막 모바일 당신이 진짜로 원했던 모험의 시작검은사막 모바일의 광활한 세계로 모험가 여러분을 초대합니다 공식 홈페이지 붉은사막 처절한 용병들의 서사시 광대한 파이웰 대륙 피로 써내려가는 생존과 투쟁의 기록 붉은사막의 세계를 만나보세요 공식 홈페이지 도깨비 세상을 흔들어 사람들의 꿈에서 힘을 얻고 함께 성장하는 도깨비 귀엽고 개성있는 도깨비들과 함께 새로운 모험을 떠나보세요 공식 홈페이지 플랜 기억에

In [9]:
type(result)

str

### 도박사이트 40개, 광고사이트 30개에서 html 한글 추출 

#### 도박사이트 40개

In [11]:
# 도박사이트 40개
df_gambling = make_df_from_url_list(unique_gambling_url_list, mode='selenium')
df_gambling.info()

1 / 40
2 / 40
3 / 40
4 / 40
5 / 40
6 / 40
7 / 40
8 / 40
9 / 40
10 / 40
11 / 40
12 / 40
13 / 40
14 / 40
15 / 40
16 / 40
17 / 40
18 / 40
19 / 40
20 / 40
21 / 40
22 / 40
23 / 40
24 / 40
25 / 40
26 / 40
27 / 40
28 / 40
29 / 40
30 / 40
31 / 40
32 / 40
33 / 40
34 / 40
35 / 40
36 / 40
37 / 40
38 / 40
39 / 40
40 / 40
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   url          40 non-null     object
 1   html_korean  40 non-null     object
dtypes: object(2)
memory usage: 768.0+ bytes


In [12]:
df_gambling

,url,html_korean
0,https://kcy-01.com/?ref=7731,곽철용 카지노 슬롯게임 라이브카지노 입금 출금 머니이동 이벤트존 공지사항 고객센터 ...
1,http://gn3300.com/login.asp,지니카지노 페이지네이션 충전 환전 게시판 게임존 게임존 로그인 진행중입니다잠시만 기...
2,https://nd-0001.com/,네임드 네임드 그만보기 그만보기 그만보기 그만보기 그만보기 그만보기 그만보기 요청 ...
3,https://onetop.pw/Main,크로스 스페셜 실시간 보글보글 보글보글 분 보글보글 분 보글보글 분 슈퍼마리오 슈퍼...
4,https://arr-365.com/,오로라 에 오신것을 환영합니다 입금신청 출금신청 경기결과 베팅내역 공지사항 이벤트 ...
5,https://lovecasino.live/,실시간입금현황 순서중요 게임서비스로 연결중입니다 카지노 슬롯 준비중입니다 미니게임 ...
6,https://wwwholdem.com/,
7,http://up-5577.com/login.asp,업카지노 페이지네이션 충전 환전 게시판 게임존 게임존 로그인 진행중입니다잠시만 기다...
8,https://yes2.bet/?referral=9090,스포츠 라이브 스포츠 카지노 미니게임 공지사항 이벤트 오후 한국어 우리와 함께하세요...
9,https://winner1.top/?regcode=7745,자바스크립트에서 사용하는 전역변수 선언 버튼 방지 오늘하루 열지않기 오늘하루 열지않...


In [13]:
df_gambling.loc[0]['html_korean']

'곽철용 카지노 슬롯게임 라이브카지노 입금 출금 머니이동 이벤트존 공지사항 고객센터 쪽지함 마이페이지 쿠키삭제 평생도메인 안내 곽철용카지노 평생도메인 안내 곽철용카지노 로그인 회원가입 로그인 회원가입 슬롯게임 카지노 점검중 공지규정 곽철용 카지노 양방베팅 및 작업베팅 기준 곽철용카지노 카지노 및 슬롯 이용시 선택 옵션 주의 안내 곽철용카지노 잭팟성 최대 배당금 제한 곽철용카지노 고객센터 텔레그램 사칭주의안내 곽철용카지노 본인인증 및 이벤트 악용안내 이벤트존 곽철용 카지노 출석 이벤트 곽철용 카지노 프리스핀데이 이벤트 곽철용 카지노 일일 주간 페이백 회원 선택 곽철용 카지노 개인 콤프롤링포인트 지급 이벤트 곽철용 카지노 커뮤니티 후기리뷰글 이벤트 곽철용카지노 관리자 곽철용카지노 입금규정 입금신청 문의 계좌확인 발급된 계좌 입금 최소 만 원부터 충전 가능합니다 입금신청 하시면 자동으로 계좌발급이 이루어지며 신청금액을 입금하시면 자동 충전이 완료됩니다 회원 가입 시 등록된 계좌로 입금된 경우에만 충전이 정상 처리됩니다 타인 명의 및 입금자명을 변경하여 입금 시 승인 불가합니다 가상 계좌가 수시로 변경됨으로 입금 전 꼭 계좌 확인 후 이체해 주셔야 합니다 입금계좌 변경으로 이체가 되지않을 경우 일반문의 또는 고객센터 텔레그램으로 문의주시면 신속히 처리가능합니다당일 출금 내역이 존재할 시 모든 보너스 지급은 제한됩니다충전 옵션을 슬롯으로 선택 후 카지노이용 혹은 반대로 이용 적발 시 충전금액이 몰수 처리되며 추후 이벤트 참여에 불이익 있을 수 있습니다입금신청시 안내사항입금점검중 보너스없음 첫충매충 입플 롤링이 적용됩니다 첫충매충입플을 받으시면 슬롯만 이용 가능합니다출금규정 일 시간 출금 신청이 가능합니다 출금 소요시간은 최소 분에서 최대분안에 처리됩니다 등록된 본인 계좌로만 출금 가능합니다 부터 은행점검시간 입금하신 금액에 맞게 롤링 충족을 하시고 게임 종료 후 바로 출금 신청시 롤링 부족으로 확인 될 수 있습니다 외부서버로 내역이 넘어오기까지 분정도 소요되는점 참고 

In [14]:
# 결측치 제거

df_gambling_filtered = df_gambling[df_gambling['html_korean'].str.len() > 0]
df_gambling_filtered = df_gambling_filtered.reset_index(drop=True)
df_gambling_filtered

,url,html_korean
0,https://kcy-01.com/?ref=7731,곽철용 카지노 슬롯게임 라이브카지노 입금 출금 머니이동 이벤트존 공지사항 고객센터 ...
1,http://gn3300.com/login.asp,지니카지노 페이지네이션 충전 환전 게시판 게임존 게임존 로그인 진행중입니다잠시만 기...
2,https://nd-0001.com/,네임드 네임드 그만보기 그만보기 그만보기 그만보기 그만보기 그만보기 그만보기 요청 ...
3,https://onetop.pw/Main,크로스 스페셜 실시간 보글보글 보글보글 분 보글보글 분 보글보글 분 슈퍼마리오 슈퍼...
4,https://arr-365.com/,오로라 에 오신것을 환영합니다 입금신청 출금신청 경기결과 베팅내역 공지사항 이벤트 ...
5,https://lovecasino.live/,실시간입금현황 순서중요 게임서비스로 연결중입니다 카지노 슬롯 준비중입니다 미니게임 ...
6,http://up-5577.com/login.asp,업카지노 페이지네이션 충전 환전 게시판 게임존 게임존 로그인 진행중입니다잠시만 기다...
7,https://yes2.bet/?referral=9090,스포츠 라이브 스포츠 카지노 미니게임 공지사항 이벤트 오후 한국어 우리와 함께하세요...
8,https://winner1.top/?regcode=7745,자바스크립트에서 사용하는 전역변수 선언 버튼 방지 오늘하루 열지않기 오늘하루 열지않...
9,http://lf-1122.com/login.asp,루피카지노 페이지네이션 충전 환전 게시판 게임존 게임존 로그인 진행중입니다잠시만 기...


In [15]:
len(df_gambling_filtered)

37

In [22]:
# .csv 파일로 저장
gambling_excel_file_name = f"gambling_url_html_korean.csv"   # 파일 이름.
# df_gambling_filtered.to_csv(gambling_excel_file_name, encoding="utf-8", index=False)  # .csv로 저장

In [20]:
# 읽기 테스트
df_test = pd.read_csv(gambling_excel_file_name, encoding='utf-8')
df_test

,url,html_korean
0,https://kcy-01.com/?ref=7731,곽철용 카지노 슬롯게임 라이브카지노 입금 출금 머니이동 이벤트존 공지사항 고객센터 ...
1,http://gn3300.com/login.asp,지니카지노 페이지네이션 충전 환전 게시판 게임존 게임존 로그인 진행중입니다잠시만 기...
2,https://nd-0001.com/,네임드 네임드 그만보기 그만보기 그만보기 그만보기 그만보기 그만보기 그만보기 요청 ...
3,https://onetop.pw/Main,크로스 스페셜 실시간 보글보글 보글보글 분 보글보글 분 보글보글 분 슈퍼마리오 슈퍼...
4,https://arr-365.com/,오로라 에 오신것을 환영합니다 입금신청 출금신청 경기결과 베팅내역 공지사항 이벤트 ...
5,https://lovecasino.live/,실시간입금현황 순서중요 게임서비스로 연결중입니다 카지노 슬롯 준비중입니다 미니게임 ...
6,http://up-5577.com/login.asp,업카지노 페이지네이션 충전 환전 게시판 게임존 게임존 로그인 진행중입니다잠시만 기다...
7,https://yes2.bet/?referral=9090,스포츠 라이브 스포츠 카지노 미니게임 공지사항 이벤트 오후 한국어 우리와 함께하세요...
8,https://winner1.top/?regcode=7745,자바스크립트에서 사용하는 전역변수 선언 버튼 방지 오늘하루 열지않기 오늘하루 열지않...
9,http://lf-1122.com/login.asp,루피카지노 페이지네이션 충전 환전 게시판 게임존 게임존 로그인 진행중입니다잠시만 기...


#### 광고사이트 30개

In [21]:
# 광고사이트 30개
df_ad = make_df_from_url_list(unique_ad_url_list)
df_ad.info()

1 / 30
2 / 30
3 / 30
4 / 30
5 / 30
6 / 30
7 / 30
8 / 30
9 / 30
10 / 30
11 / 30
12 / 30
13 / 30
14 / 30
15 / 30
16 / 30
17 / 30
18 / 30
19 / 30
20 / 30
21 / 30
22 / 30
23 / 30
24 / 30
25 / 30
26 / 30
27 / 30
28 / 30
29 / 30
30 / 30
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   url          30 non-null     object
 1   html_korean  30 non-null     object
dtypes: object(2)
memory usage: 608.0+ bytes


In [23]:
df_ad

,url,html_korean
0,https://www.kiwoometf.com/service/invest/KO040...,히어로즈 도 키움답게 운용사 키움투자자산운용 코세프 공식홈페이지 공식홈페이지 선구안...
1,https://h5.g123.jp/game/arifure?lang=ko&platfo...,흔해빠진 직업으로 세계최강 리벨리온 소울 게임 시작 흔해빠진 직업으로 세계최강 리벨...
2,https://h5.g123.jp/game/arifure?lang=ko&platfo...,흔해빠진 직업으로 세계최강 리벨리온 소울 게임 시작 흔해빠진 직업으로 세계최강 리벨...
3,https://lineagem.plaync.com/preorder/record/ho...,리니지 업데이트 사전예약 진행 중 리니지 리니지엠 린엠 리니지 리니지사전예약페이지조...
4,https://www.bjmart.kr/shop_view/?idx=235&utm_s...,나라모 더드림행사 박스 포 나라모 풍성한 자신감 나라모 더드림행사 박스 포 나라모 ...
5,https://www.miral.org/bbs/view.asp?bbsIdx=9038...,밀알복지재단 누구나데이터 오늘의리포트 밀알복지재단은 년 설립되어 장애인의 완전한 사...
6,https://soc.seaofconquest.com/pc/pc-landing1-k...,정복의 바다 기프트 코드 쿠키 사용자 정의 웹사이트를 방문할 때 브라우저는 쿠키 형...
7,https://hanghae99.spartacodingclub.kr/plus/be?...,개발자 커리어 개척 캠프 항해 스파르타코딩클럽 코딩 부트캠프 항해에 합류하세요 일 ...
8,https://digitalshinhansec.com/global/event01/?...,메타 태그 신한투자증권 계좌 첫 개설시 인기 미국주식 지급 신한투자증권 웹 모바일 ...
9,https://www.bjmart.kr/shop_view/?idx=235&utm_s...,나라모 더드림행사 박스 포 나라모 풍성한 자신감 나라모 더드림행사 박스 포 나라모 ...


In [25]:
# .csv 파일로 저장
ad_excel_file_name = f"ad_url_html_korean.csv"   # 파일 이름.
df_ad.to_csv(ad_excel_file_name, encoding="utf-8", index=False)  # .csv로 저장

In [26]:
# 읽기 테스트
df_test = pd.read_csv(ad_excel_file_name, encoding='utf-8')
df_test

,url,html_korean
0,https://www.kiwoometf.com/service/invest/KO040...,히어로즈 도 키움답게 운용사 키움투자자산운용 코세프 공식홈페이지 공식홈페이지 선구안...
1,https://h5.g123.jp/game/arifure?lang=ko&platfo...,흔해빠진 직업으로 세계최강 리벨리온 소울 게임 시작 흔해빠진 직업으로 세계최강 리벨...
2,https://h5.g123.jp/game/arifure?lang=ko&platfo...,흔해빠진 직업으로 세계최강 리벨리온 소울 게임 시작 흔해빠진 직업으로 세계최강 리벨...
3,https://lineagem.plaync.com/preorder/record/ho...,리니지 업데이트 사전예약 진행 중 리니지 리니지엠 린엠 리니지 리니지사전예약페이지조...
4,https://www.bjmart.kr/shop_view/?idx=235&utm_s...,나라모 더드림행사 박스 포 나라모 풍성한 자신감 나라모 더드림행사 박스 포 나라모 ...
5,https://www.miral.org/bbs/view.asp?bbsIdx=9038...,밀알복지재단 누구나데이터 오늘의리포트 밀알복지재단은 년 설립되어 장애인의 완전한 사...
6,https://soc.seaofconquest.com/pc/pc-landing1-k...,정복의 바다 기프트 코드 쿠키 사용자 정의 웹사이트를 방문할 때 브라우저는 쿠키 형...
7,https://hanghae99.spartacodingclub.kr/plus/be?...,개발자 커리어 개척 캠프 항해 스파르타코딩클럽 코딩 부트캠프 항해에 합류하세요 일 ...
8,https://digitalshinhansec.com/global/event01/?...,메타 태그 신한투자증권 계좌 첫 개설시 인기 미국주식 지급 신한투자증권 웹 모바일 ...
9,https://www.bjmart.kr/shop_view/?idx=235&utm_s...,나라모 더드림행사 박스 포 나라모 풍성한 자신감 나라모 더드림행사 박스 포 나라모 ...


## 토큰화
- konlpy로 토큰화 진행함
- pycharm 환경에서 토큰화 후 작업

In [80]:
# 파이썬으로 토큰화한 파일 불러오기
gambling_tokenized_file_name = "gambling_tokenized_html_korean.csv"
df_gambling_tokenized = pd.read_csv(gambling_tokenized_file_name, encoding='utf-8')
df_gambling_tokenized

,url,html_korean,tokens
0,https://kcy-01.com/?ref=7731,곽철용 카지노 슬롯게임 라이브카지노 입금 출금 머니이동 이벤트존 공지사항 고객센터 ...,"['곽철', '카지노', '슬롯', '게임', '라이브', '카지노', '입금', ..."
1,http://gn3300.com/login.asp,지니카지노 페이지네이션 충전 환전 게시판 게임존 게임존 로그인 진행중입니다잠시만 기...,"['지니', '카지노', '페이지', '네이션', '충전', '환전', '게시판',..."
2,https://nd-0001.com/,네임드 네임드 그만보기 그만보기 그만보기 그만보기 그만보기 그만보기 그만보기 요청 ...,"['네임', '네임', '요청', '주소', '경우', '동기', '요청', '변경..."
3,https://onetop.pw/Main,크로스 스페셜 실시간 보글보글 보글보글 분 보글보글 분 보글보글 분 슈퍼마리오 슈퍼...,"['크로스', '스페셜', '실시간', '슈퍼', '마리오', '슈퍼', '슈퍼',..."
4,https://arr-365.com/,오로라 에 오신것을 환영합니다 입금신청 출금신청 경기결과 베팅내역 공지사항 이벤트 ...,"['오로라', '환영', '입금', '신청', '출금', '신청', '경기', '결..."
5,https://lovecasino.live/,실시간입금현황 순서중요 게임서비스로 연결중입니다 카지노 슬롯 준비중입니다 미니게임 ...,"['실시간', '입금', '현황', '순서', '중요', '게임', '서비스', '..."
6,http://up-5577.com/login.asp,업카지노 페이지네이션 충전 환전 게시판 게임존 게임존 로그인 진행중입니다잠시만 기다...,"['카지노', '페이지', '네이션', '충전', '환전', '게시판', '게임',..."
7,https://yes2.bet/?referral=9090,스포츠 라이브 스포츠 카지노 미니게임 공지사항 이벤트 오후 한국어 우리와 함께하세요...,"['스포츠', '라이브', '스포츠', '카지노', '미니게임', '공지', '사항..."
8,https://winner1.top/?regcode=7745,자바스크립트에서 사용하는 전역변수 선언 버튼 방지 오늘하루 열지않기 오늘하루 열지않...,"['자바', '스크립트', '사용', '전역', '변수', '선언', '버튼', '..."
9,http://lf-1122.com/login.asp,루피카지노 페이지네이션 충전 환전 게시판 게임존 게임존 로그인 진행중입니다잠시만 기...,"['루피', '카지노', '페이지', '네이션', '충전', '환전', '게시판',..."


In [81]:
ad_tokenized_file_name = "ad_tokenized_html_korean.csv"
df_ad_tokenized = pd.read_csv(ad_tokenized_file_name, encoding='utf-8')
df_ad_tokenized

,url,html_korean,tokens
0,https://www.kiwoometf.com/service/invest/KO040...,히어로즈 도 키움답게 운용사 키움투자자산운용 코세프 공식홈페이지 공식홈페이지 선구안...,"['히어로즈', '키움', '운용', '키움', '투자', '자산운용', '코세프'..."
1,https://h5.g123.jp/game/arifure?lang=ko&platfo...,흔해빠진 직업으로 세계최강 리벨리온 소울 게임 시작 흔해빠진 직업으로 세계최강 리벨...,"['직업', '세계', '최강', '리벨리온', '소울', '게임', '시작', '..."
2,https://h5.g123.jp/game/arifure?lang=ko&platfo...,흔해빠진 직업으로 세계최강 리벨리온 소울 게임 시작 흔해빠진 직업으로 세계최강 리벨...,"['직업', '세계', '최강', '리벨리온', '소울', '게임', '시작', '..."
3,https://lineagem.plaync.com/preorder/record/ho...,리니지 업데이트 사전예약 진행 중 리니지 리니지엠 린엠 리니지 리니지사전예약페이지조...,"['리니지', '업데이트', '사전예약', '진행', '리니지', '리니지', '리..."
4,https://www.bjmart.kr/shop_view/?idx=235&utm_s...,나라모 더드림행사 박스 포 나라모 풍성한 자신감 나라모 더드림행사 박스 포 나라모 ...,"['나라', '더드림', '행사', '박스', '자신감', '나라', '더드림', ..."
5,https://www.miral.org/bbs/view.asp?bbsIdx=9038...,밀알복지재단 누구나데이터 오늘의리포트 밀알복지재단은 년 설립되어 장애인의 완전한 사...,"['밀알', '복지', '재단', '누구', '데이터', '오늘', '리포트', '..."
6,https://soc.seaofconquest.com/pc/pc-landing1-k...,정복의 바다 기프트 코드 쿠키 사용자 정의 웹사이트를 방문할 때 브라우저는 쿠키 형...,"['정복', '바다', '기프트', '코드', '쿠키', '사용', '정의', '사..."
7,https://hanghae99.spartacodingclub.kr/plus/be?...,개발자 커리어 개척 캠프 항해 스파르타코딩클럽 코딩 부트캠프 항해에 합류하세요 일 ...,"['개발자', '커리어', '개척', '캠프', '항해', '스파르타', '코딩',..."
8,https://digitalshinhansec.com/global/event01/?...,메타 태그 신한투자증권 계좌 첫 개설시 인기 미국주식 지급 신한투자증권 웹 모바일 ...,"['메타', '태그', '신한', '투자', '증권', '계좌', '개설', '인기..."
9,https://www.bjmart.kr/shop_view/?idx=235&utm_s...,나라모 더드림행사 박스 포 나라모 풍성한 자신감 나라모 더드림행사 박스 포 나라모 ...,"['나라', '더드림', '행사', '박스', '자신감', '나라', '더드림', ..."


## XGBOOST

In [66]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

### 1. 데이터 준비:

tokens 컬럼을 문자열로 변환: 

XGBoost는 문자열 형태의 입력을 직접 처리할 수 없으므로, 
tokens 컬럼의 리스트를 다시 문자열로 변환해야 합니다. 
이 때, 토큰들을 공백으로 연결하여 하나의 문자열로 만들 수 있습니다.

타겟 변수 준비: 분류 모델의 타겟 변수(클래스 레이블)를 준비합니다. 예를 들어, 도박 사이트/광고 사이트 여부를 나타내는 컬럼이 필요합니다.

데이터 분할: 전체 데이터를 학습 데이터와 테스트 데이터로 분할합니다.

In [67]:
df_gambling_tokenized['text'] = df_gambling_tokenized['tokens'].apply(lambda x: ' '.join(x) if isinstance(x, list) else '')

In [68]:
df_gambling_tokenized.head()

,url,html_korean,tokens,text
0,https://kcy-01.com/?ref=7731,곽철용 카지노 슬롯게임 라이브카지노 입금 출금 머니이동 이벤트존 공지사항 고객센터 ...,"['곽철', '카지노', '슬롯', '게임', '라이브', '카지노', '입금', ...",
1,http://gn3300.com/login.asp,지니카지노 페이지네이션 충전 환전 게시판 게임존 게임존 로그인 진행중입니다잠시만 기...,"['지니', '카지노', '페이지', '네이션', '충전', '환전', '게시판',...",
2,https://nd-0001.com/,네임드 네임드 그만보기 그만보기 그만보기 그만보기 그만보기 그만보기 그만보기 요청 ...,"['네임', '네임', '요청', '주소', '경우', '동기', '요청', '변경...",
3,https://onetop.pw/Main,크로스 스페셜 실시간 보글보글 보글보글 분 보글보글 분 보글보글 분 슈퍼마리오 슈퍼...,"['크로스', '스페셜', '실시간', '슈퍼', '마리오', '슈퍼', '슈퍼',...",
4,https://arr-365.com/,오로라 에 오신것을 환영합니다 입금신청 출금신청 경기결과 베팅내역 공지사항 이벤트 ...,"['오로라', '환영', '입금', '신청', '출금', '신청', '경기', '결...",


In [72]:
type(df_gambling_tokenized['tokens'][0])

str

In [77]:
import re

def preprocess_text(text):
  if isinstance(text, str):
    # 한글과 공백 문자만 남기고 나머지는 제거
    text = re.sub(r"[^가-힣\s]", "", text)
    return text
  else:
    return "" 

In [78]:
input_str = "['곽철', '카지노', '슬롯', '게임', '라이브']"
result = preprocess_text(input_str)
result

'곽철 카지노 슬롯 게임 라이브'

In [97]:
df_gambling_tokenized['tokens_str'] = df_gambling_tokenized['tokens'].apply(preprocess_text)
df_gambling_tokenized.head()

,url,html_korean,tokens,tokens_str,label
0,https://kcy-01.com/?ref=7731,곽철용 카지노 슬롯게임 라이브카지노 입금 출금 머니이동 이벤트존 공지사항 고객센터 ...,"['곽철', '카지노', '슬롯', '게임', '라이브', '카지노', '입금', ...",곽철 카지노 슬롯 게임 라이브 카지노 입금 출금 머니 이동 이벤트 공지 사항 고객 ...,gambling
1,http://gn3300.com/login.asp,지니카지노 페이지네이션 충전 환전 게시판 게임존 게임존 로그인 진행중입니다잠시만 기...,"['지니', '카지노', '페이지', '네이션', '충전', '환전', '게시판',...",지니 카지노 페이지 네이션 충전 환전 게시판 게임 게임 로그인 진행 아이디 입력 비...,gambling
2,https://nd-0001.com/,네임드 네임드 그만보기 그만보기 그만보기 그만보기 그만보기 그만보기 그만보기 요청 ...,"['네임', '네임', '요청', '주소', '경우', '동기', '요청', '변경...",네임 네임 요청 주소 경우 동기 요청 변경 전송 데이터 존재 코드 마감 코드 정상 ...,gambling
3,https://onetop.pw/Main,크로스 스페셜 실시간 보글보글 보글보글 분 보글보글 분 보글보글 분 슈퍼마리오 슈퍼...,"['크로스', '스페셜', '실시간', '슈퍼', '마리오', '슈퍼', '슈퍼',...",크로스 스페셜 실시간 슈퍼 마리오 슈퍼 슈퍼 슈퍼 룰렛 묵찌빠 룰렛 묵찌빠 룰렛 묵...,gambling
4,https://arr-365.com/,오로라 에 오신것을 환영합니다 입금신청 출금신청 경기결과 베팅내역 공지사항 이벤트 ...,"['오로라', '환영', '입금', '신청', '출금', '신청', '경기', '결...",오로라 환영 입금 신청 출금 신청 경기 결과 베팅 내역 공지 사항 이벤트 쪽지 고객...,gambling


In [109]:
df_gambling_tokenized['label'] = 0
df_gambling_tokenized.head()

,url,html_korean,tokens,tokens_str,label
0,https://kcy-01.com/?ref=7731,곽철용 카지노 슬롯게임 라이브카지노 입금 출금 머니이동 이벤트존 공지사항 고객센터 ...,"['곽철', '카지노', '슬롯', '게임', '라이브', '카지노', '입금', ...",곽철 카지노 슬롯 게임 라이브 카지노 입금 출금 머니 이동 이벤트 공지 사항 고객 ...,0
1,http://gn3300.com/login.asp,지니카지노 페이지네이션 충전 환전 게시판 게임존 게임존 로그인 진행중입니다잠시만 기...,"['지니', '카지노', '페이지', '네이션', '충전', '환전', '게시판',...",지니 카지노 페이지 네이션 충전 환전 게시판 게임 게임 로그인 진행 아이디 입력 비...,0
2,https://nd-0001.com/,네임드 네임드 그만보기 그만보기 그만보기 그만보기 그만보기 그만보기 그만보기 요청 ...,"['네임', '네임', '요청', '주소', '경우', '동기', '요청', '변경...",네임 네임 요청 주소 경우 동기 요청 변경 전송 데이터 존재 코드 마감 코드 정상 ...,0
3,https://onetop.pw/Main,크로스 스페셜 실시간 보글보글 보글보글 분 보글보글 분 보글보글 분 슈퍼마리오 슈퍼...,"['크로스', '스페셜', '실시간', '슈퍼', '마리오', '슈퍼', '슈퍼',...",크로스 스페셜 실시간 슈퍼 마리오 슈퍼 슈퍼 슈퍼 룰렛 묵찌빠 룰렛 묵찌빠 룰렛 묵...,0
4,https://arr-365.com/,오로라 에 오신것을 환영합니다 입금신청 출금신청 경기결과 베팅내역 공지사항 이벤트 ...,"['오로라', '환영', '입금', '신청', '출금', '신청', '경기', '결...",오로라 환영 입금 신청 출금 신청 경기 결과 베팅 내역 공지 사항 이벤트 쪽지 고객...,0


In [99]:
df_ad_tokenized['tokens_str'] = df_ad_tokenized['tokens'].apply(preprocess_text)
df_ad_tokenized.head()

,url,html_korean,tokens,tokens_str,label
0,https://www.kiwoometf.com/service/invest/KO040...,히어로즈 도 키움답게 운용사 키움투자자산운용 코세프 공식홈페이지 공식홈페이지 선구안...,"['히어로즈', '키움', '운용', '키움', '투자', '자산운용', '코세프'...",히어로즈 키움 운용 키움 투자 자산운용 코세프 공식 홈페이지 공식 홈페이지 선구안 ...,ad
1,https://h5.g123.jp/game/arifure?lang=ko&platfo...,흔해빠진 직업으로 세계최강 리벨리온 소울 게임 시작 흔해빠진 직업으로 세계최강 리벨...,"['직업', '세계', '최강', '리벨리온', '소울', '게임', '시작', '...",직업 세계 최강 리벨리온 소울 게임 시작 직업 세계 최강 리벨리온 소울 게임 시작 ...,ad
2,https://h5.g123.jp/game/arifure?lang=ko&platfo...,흔해빠진 직업으로 세계최강 리벨리온 소울 게임 시작 흔해빠진 직업으로 세계최강 리벨...,"['직업', '세계', '최강', '리벨리온', '소울', '게임', '시작', '...",직업 세계 최강 리벨리온 소울 게임 시작 직업 세계 최강 리벨리온 소울 게임 시작 ...,ad
3,https://lineagem.plaync.com/preorder/record/ho...,리니지 업데이트 사전예약 진행 중 리니지 리니지엠 린엠 리니지 리니지사전예약페이지조...,"['리니지', '업데이트', '사전예약', '진행', '리니지', '리니지', '리...",리니지 업데이트 사전예약 진행 리니지 리니지 리니지 리니지 사전예약 페이지 조회 도...,ad
4,https://www.bjmart.kr/shop_view/?idx=235&utm_s...,나라모 더드림행사 박스 포 나라모 풍성한 자신감 나라모 더드림행사 박스 포 나라모 ...,"['나라', '더드림', '행사', '박스', '자신감', '나라', '더드림', ...",나라 더드림 행사 박스 자신감 나라 더드림 행사 박스 자신감 나라 더드림 행사 박스...,ad


In [110]:
df_ad_tokenized['label'] = 1
df_ad_tokenized.head()

,url,html_korean,tokens,tokens_str,label
0,https://www.kiwoometf.com/service/invest/KO040...,히어로즈 도 키움답게 운용사 키움투자자산운용 코세프 공식홈페이지 공식홈페이지 선구안...,"['히어로즈', '키움', '운용', '키움', '투자', '자산운용', '코세프'...",히어로즈 키움 운용 키움 투자 자산운용 코세프 공식 홈페이지 공식 홈페이지 선구안 ...,1
1,https://h5.g123.jp/game/arifure?lang=ko&platfo...,흔해빠진 직업으로 세계최강 리벨리온 소울 게임 시작 흔해빠진 직업으로 세계최강 리벨...,"['직업', '세계', '최강', '리벨리온', '소울', '게임', '시작', '...",직업 세계 최강 리벨리온 소울 게임 시작 직업 세계 최강 리벨리온 소울 게임 시작 ...,1
2,https://h5.g123.jp/game/arifure?lang=ko&platfo...,흔해빠진 직업으로 세계최강 리벨리온 소울 게임 시작 흔해빠진 직업으로 세계최강 리벨...,"['직업', '세계', '최강', '리벨리온', '소울', '게임', '시작', '...",직업 세계 최강 리벨리온 소울 게임 시작 직업 세계 최강 리벨리온 소울 게임 시작 ...,1
3,https://lineagem.plaync.com/preorder/record/ho...,리니지 업데이트 사전예약 진행 중 리니지 리니지엠 린엠 리니지 리니지사전예약페이지조...,"['리니지', '업데이트', '사전예약', '진행', '리니지', '리니지', '리...",리니지 업데이트 사전예약 진행 리니지 리니지 리니지 리니지 사전예약 페이지 조회 도...,1
4,https://www.bjmart.kr/shop_view/?idx=235&utm_s...,나라모 더드림행사 박스 포 나라모 풍성한 자신감 나라모 더드림행사 박스 포 나라모 ...,"['나라', '더드림', '행사', '박스', '자신감', '나라', '더드림', ...",나라 더드림 행사 박스 자신감 나라 더드림 행사 박스 자신감 나라 더드림 행사 박스...,1


#### 학습 데이터 만들기

In [116]:
df_gambling_label = df_gambling_tokenized[['tokens_str', 'label']]
df_gambling_label.head(1)

,tokens_str,label
0,곽철 카지노 슬롯 게임 라이브 카지노 입금 출금 머니 이동 이벤트 공지 사항 고객 ...,0


In [117]:
df_ad_label = df_ad_tokenized[['tokens_str', 'label']]
df_ad_label.head(1)

,tokens_str,label
0,히어로즈 키움 운용 키움 투자 자산운용 코세프 공식 홈페이지 공식 홈페이지 선구안 ...,1


In [118]:
# 행 방향으로 연결
df_combined = pd.concat([df_gambling_label, df_ad_label], ignore_index=True)
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tokens_str  67 non-null     object
 1   label       67 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.2+ KB


In [119]:
df_combined.head()

,tokens_str,label
0,곽철 카지노 슬롯 게임 라이브 카지노 입금 출금 머니 이동 이벤트 공지 사항 고객 ...,0
1,지니 카지노 페이지 네이션 충전 환전 게시판 게임 게임 로그인 진행 아이디 입력 비...,0
2,네임 네임 요청 주소 경우 동기 요청 변경 전송 데이터 존재 코드 마감 코드 정상 ...,0
3,크로스 스페셜 실시간 슈퍼 마리오 슈퍼 슈퍼 슈퍼 룰렛 묵찌빠 룰렛 묵찌빠 룰렛 묵...,0
4,오로라 환영 입금 신청 출금 신청 경기 결과 베팅 내역 공지 사항 이벤트 쪽지 고객...,0


### 2. 텍스트 데이터 벡터화:

TF-IDF 벡터화: 텍스트 데이터를 수치형 벡터로 변환하기 위해 TF-IDF (Term Frequency-Inverse Document Frequency) 벡터화 방법을 사용합니다.

TfidfVectorizer를 사용하여 텍스트 데이터를 TF-IDF 벡터로 변환합니다.

fit_transform() 메서드를 사용하여 학습 데이터를 벡터화하고, transform() 메서드를 사용하여 테스트 데이터를 벡터화합니다.

In [120]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd

# 데이터프레임 df_combined가 이미 존재한다고 가정

# 1. 데이터 분할 (먼저 분할해야 TF-IDF 벡터라이저를 학습 데이터에만 fit 할 수 있습니다)
X = df_combined['tokens_str']
y = df_combined['label']  # label 컬럼이 있다고 가정

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 2. TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer()

# 학습 데이터에 대해 fit_transform() 수행
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# 테스트 데이터에 대해 transform() 수행
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# 결과 확인
print("학습 데이터 TF-IDF 벡터 크기:", X_train_tfidf.shape)
print("테스트 데이터 TF-IDF 벡터 크기:", X_test_tfidf.shape)

학습 데이터 TF-IDF 벡터 크기: (53, 5347)
테스트 데이터 TF-IDF 벡터 크기: (14, 5347)


### 3. XGBoost 모델 학습:

XGBoost 모델 초기화: XGBClassifier를 사용하여 XGBoost 분류 모델을 초기화합니다.

모델 학습: fit() 메서드를 사용하여 학습 데이터를 이용하여 모델을 학습합니다.

In [121]:
from xgboost import XGBClassifier

# XGBoost 모델 초기화
xgb_classifier = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# 모델 학습
xgb_classifier.fit(X_train_tfidf, y_train)

C:\Users\USER\.conda\envs\tc_env\lib\site-packages\xgboost\core.py:158: UserWarning: [23:59:15] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

### 4. 모델 평가:

테스트 데이터 예측: predict() 메서드를 사용하여 테스트 데이터에 대한 예측을 수행합니다.

평가 지표 계산: 정확도, 정밀도, 재현율, F1 점수 등을 사용하여 모델의 성능을 평가합니다.

In [123]:
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# 데이터프레임 df_combined가 이미 존재하고, TF-IDF 벡터화 및 모델 학습이 완료되었다고 가정

# 레이블 인코딩 (문자열 레이블을 숫자형으로 변환)
# df_combined['label'] = df_combined['label'].map({'gambling': 1, 'normal': 0})  # 이 코드는 이미 실행되었다고 가정

# 1. 데이터 분할 (TF-IDF 벡터화 이전에 수행해야 함)
X = df_combined['tokens_str']
y = df_combined['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# y_train과 y_test의 데이터 타입을 확인하고, 문자열인 경우 숫자형으로 변환
# print("y_train 데이터 타입 (변환 전):", y_train.dtype)
# print("y_test 데이터 타입 (변환 전):", y_test.dtype)

y_train = y_train.astype(int)
y_test = y_test.astype(int)

# print("y_train 데이터 타입 (변환 후):", y_train.dtype)
# print("y_test 데이터 타입 (변환 후):", y_test.dtype)


# 2. TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# 3. XGBoost 모델 학습
# XGBoost 모델 초기화
xgb_classifier = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# 모델 학습
xgb_classifier.fit(X_train_tfidf, y_train)

# 4. 모델 평가
# 테스트 데이터 예측
y_pred = xgb_classifier.predict(X_test_tfidf)

# 평가 지표 계산
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

Accuracy: 0.9285714285714286
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.88      0.93         8
           1       0.86      1.00      0.92         6

    accuracy                           0.93        14
   macro avg       0.93      0.94      0.93        14
weighted avg       0.94      0.93      0.93        14



C:\Users\USER\.conda\envs\tc_env\lib\site-packages\xgboost\core.py:158: UserWarning: [00:00:56] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [124]:
# 학습된 모델 저장
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import xgboost as xgb  


# 모델 저장
model_filename = "xgboost_model.json"  
xgb_classifier.save_model(model_filename)
print("모델 저장 완료")

모델 저장 완료
